In [2]:
# GRU with Minimal Gated Units
import numpy as np
import os.path

# Seed random
np.random.seed(0)

# Read data and setup maps for integer encoding and decoding.
data = open('input.txt', 'r').read()
chars = sorted(list(set(data))) # Sort makes model predictable (if seeded).
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # dict from enumerate: char and index
ix_to_char = { i:ch for i,ch in enumerate(chars) } # dict from enumerate: index and char

# Activation functions
# NOTE: Derivatives are calculated using outcomes of their primitives (which are already calculated during forward prop).
def sigmoid(input, deriv=False):
    if deriv:
        return input*(1-input) # sig' = sig(1-sig) - only if claculated in forward prop already!
    else:
        return 1 / (1 + np.exp(-input))

def tanh(input, deriv=False):
    if deriv:
        return 1 - input ** 2 # tanh'=1-tanh**2
    else:
        return np.tanh(input)

# Derivative is directly calculated in backprop (in combination with cross-entropy loss function).
def softmax(input):
    # Subtraction of max value improves numerical stability.
    e_input = np.exp(input - np.max(input))
    return e_input / e_input.sum()

# Hyper parameters
N, h_size, o_size = vocab_size, vocab_size, vocab_size # Hidden size is set to vocab_size, assuming that level of abstractness is approximately proportional to vocab_size (but can be set to any other value).
seq_length = 5 # Longer sequence lengths allow for lengthier latent dependencies to be trained.
learning_rate = 1e-1

# Model parameter initialization
#if os.path.isfile("gru_weights.data"):
    #initfile = open("gru_weights.data", "rb")
    #init_data = np.load(initfile)
    #Wy = init_data['Wy']
    #Wh = init_data['Wh']
    #Wr = init_data['Wr']
    #Wz = init_data['Wz']
    #Uh = init_data['Uh']
    #Ur = init_data['Ur']
    #Uz = init_data['Uz']
    #by = init_data['by']
    #bh = init_data['bh']
    #br = init_data['br']
    #bz = init_data['bz']
    #initfile.close()
#else:
Wz = np.random.rand(h_size, N) * 0.1 - 0.05
Uz = np.random.rand(h_size, h_size) * 0.1 - 0.05
bz = np.zeros((h_size, 1))

Wr = np.random.rand(h_size, N) * 0.1 - 0.05
Ur = np.random.rand(h_size, h_size) * 0.1 - 0.05
br = np.zeros((h_size, 1))

Wh = np.random.rand(h_size, N) * 0.1 - 0.05
Uh = np.random.rand(h_size, h_size) * 0.1 - 0.05
bh = np.zeros((h_size, 1))

Wy = np.random.rand(o_size, h_size) * 0.1 - 0.05
by = np.zeros((o_size, 1))

def lossFun(inputs, targets, hprev):
    # Initialize variables
    x, z, r, h_hat, h, y, p = {}, {}, {}, {}, {-1: hprev}, {}, {} # Dictionaries contain variables for each timestep.
    sequence_loss = 0

    # Forward prop
    for t in range(len(inputs)):
        # Set up one-hot encoded input
        x[t] = np.zeros((vocab_size, 1)) # for each time step t a one-hot over vocabulary size - init with all 0
        x[t][inputs[t]] = 1 # set t'th input to one (current word in one-hot)
        
        # Calculate update and reset gates
        z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + bz) # update gate
        r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + br) # reset gate
        
        # Calculate hidden units
        h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + bh)
        h[t] = np.multiply(z[t], h[t-1]) + np.multiply((1 - z[t]), h_hat[t]) # sometimes denoted s[t]
        
        # Regular output unit
        y[t] = np.dot(Wy, h[t]) + by
        
        # Probability distribution
        p[t] = softmax(y[t])
        
        # Cross-entropy loss
        # targets[t]'s entry of p[t]: since the output is 0 or 1 only one entry contributes (and yj*log(pj) becomes just log(pj))
        #loss = -np.sum(np.log(p[t][targets[t]])) # dict p: or time t (=one-hot position), whats the corresponsing target value?
        loss = -np.log(p[t][targets[t]])
        sequence_loss += loss

    # Parameter gradient initialization
    dWy, dWh, dWr, dWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
    dUh, dUr, dUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
    dby, dbh, dbr, dbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
    dhnext = np.zeros_like(h[0])
    
    # Backward prop
    for t in reversed(range(len(inputs))): # index t counting down
        # ∂loss/∂y
        dy = np.copy(p[t]) # copy output
        dy[targets[t]] -= 1 # the current target (truth) is 1 for the current t (an 0 for all other t's)
        
        # ∂loss/∂Wy and ∂loss/∂by
        dWy += np.dot(dy, h[t].T) # weight updates: Wy -> Wy - etha * d loss / dWy -> dWy += etha * d loss / dWy
        dby += dy # weight for bias is just 1
        
        # Intermediary derivatives
        dh = np.dot(Wy.T, dy) + dhnext
        dh_hat = np.multiply(dh, (1 - z[t]))
        dh_hat_l = dh_hat * tanh(h_hat[t], deriv=True) # = Wy*dy*(1-zt) * (1-tanh**2(h_hat))
        
        # ∂loss/∂Wh, ∂loss/∂Uh and ∂loss/∂bh
        dWh += np.dot(dh_hat_l, x[t].T) # Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * xt
        dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T) # Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * (rt*ht-1)
        dbh += dh_hat_l # weight for bias is just 1
        
        # Intermediary derivatives
        drhp = np.dot(Uh.T, dh_hat_l) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))
        dr = np.multiply(drhp, h[t-1]) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat)) * ht-1
        dr_l = dr * sigmoid(r[t], deriv=True) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*ht-1*sig(rt)*(1-sig(rt))
        
        # ∂loss/∂Wr, ∂loss/∂Ur and ∂loss/∂br
        dWr += np.dot(dr_l, x[t].T) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))*xt
        dUr += np.dot(dr_l, h[t-1].T) # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))*ht-1
        dbr += dr_l # # Uh * Wy*dy*(1-zt) * (1-tanh**2(h_hat))*sig(rt)*(1-sig(rt))
        
        # Intermediary derivatives
        dz = np.multiply(dh, h[t-1] - h_hat[t]) # dh * (ht-1 - h_hatt)
        dz_l = dz * sigmoid(z[t], deriv=True) # dz * sig(zt)*(1-sig(zt))
        
        # ∂loss/∂Wz, ∂loss/∂Uz and ∂loss/∂bz
        dWz += np.dot(dz_l, x[t].T) # dz * sig(zt)*(1-sig(zt))*xt
        dUz += np.dot(dz_l, h[t-1].T) # # dz * sig(zt)*(1-sig(zt))*ht-1
        dbz += dz_l # # dz * sig(zt)*(1-sig(zt))
        
        # All influences of previous layer to loss
        dh_fz_inner = np.dot(Uz.T, dz_l) # Uz * dz * sig(zt)*(1-sig(zt))
        dh_fz = np.multiply(dh, z[t]) # dh * zt
        dh_fhh = np.multiply(drhp, r[t]) # Uh * Wy*dy*(1-zt) * (1-tanh**2(Wy*dy*(1-zt))) * rt
        dh_fr = np.dot(Ur.T, dr_l) # Ur * Uh*Wy*dy*(1-zt)*ht-1*sig(rt)*(1-sig(rt))
        
        # ∂loss/∂h𝑡₋₁
        dhnext = dh_fz_inner + dh_fz + dh_fhh + dh_fr

    return sequence_loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, h[len(inputs) - 1]

def sample(h, seed_ix, n):
    # Initialize first word of sample ('seed') as one-hot encoded vector.
    x = np.zeros((vocab_size, 1)) # np.zeros(shape) -> (vocab_size, 1) = (rows, cols)
    x[seed_ix] = 1
    ixes = [seed_ix]
    
    for t in range(n):
        # Calculate update and reset gates
        z = sigmoid(np.dot(Wz, x) + np.dot(Uz, h) + bz)
        r = sigmoid(np.dot(Wr, x) + np.dot(Ur, h) + br)
        
        # Calculate hidden units
        h_hat = tanh(np.dot(Wh, x) + np.dot(Uh, np.multiply(r, h)) + bh)
        h = np.multiply(z, h) + np.multiply((1 - z), h_hat)
        
        # Regular output unit
        y = np.dot(Wy, h) + by
        
        # Probability distribution
        p = softmax(y)

        # Choose next char according to the distribution
        ix = np.random.choice(range(vocab_size), p=p.ravel()) # ravel returns flattened array, P are the probabilities for choice
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

# Initialize sampling parameters and memory gradients (for adagrad)
n, p = 0, 0
mdWy, mdWh, mdWr, mdWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
mdUh, mdUr, mdUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
mdby, mdbh, mdbr, mdbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

print_interval = 100
n_runs = 1000

while True:
    # Reset memory if appropriate
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h_size, 1))
        p = 0 # current position
    
    # Get input and target sequence - each an index list of characters
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] # current position to sequence length
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] # current position+1 to sequence length+1 (next state)

    # Occasionally sample from model and print result
    if n % print_interval == 0:
        sample_ix = sample(hprev, inputs[0], 1000)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt, ))

    # Get gradients for current model based on input and target sequences
    loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Occasionally print loss information
    if n % print_interval == 0:
        print('iter %d, loss: %f, smooth loss: %f' % (n, loss, smooth_loss))

    # Update model with adagrad (stochastic) gradient descent
    for param, dparam, mem in zip([Wy,  Wh,  Wr,  Wz,  Uh,  Ur,  Uz,  by,  bh,  br,  bz], # zip combines iterables
                                  [dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz],
                                  [mdWy,mdWh,mdWr,mdWz,mdUh,mdUr,mdUz,mdby,mdbh,mdbr,mdbz]):
        np.clip(dparam, -5, 5, out=dparam) # limit values of array - here the changes (dparam has the dWy,...)
        mem += dparam * dparam # store the squared change (gradient)
        # now actually change the model parameters
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # Small added term for numerical stability

    # Prepare for next iteration
    p += seq_length
    n += 1
    
    # check exit
    if n>n_runs:
        break
        
# save weights
#outfile = open("gru_weights.data", "wb")
#outdata = dict()
#outdata['Wy'] = Wy
#outdata['Wh'] = Wh
#outdata['Wr'] = Wr
#outdata['Wz'] = Wz
#outdata['Uh'] = Uh
#outdata['Ur'] = Ur
#outdata['Uz'] = Uz
#outdata['by'] = by
#outdata['bh'] = bh
#outdata['br'] = br
#outdata['bz'] = bz
#np.savez(outfile, **outdata)
#outfile.close()

data has 576 characters, 7 unique.
----
a e cbdfd cccda bfafccfcebfddbedaaefcdfcfdefeadb bae fbb abf ffcbad  c aaa   dffbadcf ebadadcbcbdbbef abefffbfa fad efabbeaacbeacfdffedffc bab dbffcadfbcacbacbbcebff ac bedddabdfca d bcf b bdaa dddf dddf decbe ccd abdfd ffcfcfeff abfbafcfadc da   fefdcbfbcbfabd dcaacbcbf  bddbfbedeffedba accef  deeffbbaddf cdaaf c cfbf abbdbddbcd edec bbb e dddf f cbcba ceba  a a  eacecaaccbeef fbfdfebcbbacdac a f ad fefcbcfcafb abfccbfffbfedaaafbccee bee ab b cad ffbaac dbbedfbfebdbf aabecc b afafead edecf beba ecfebbcfe dfdac bffabdfaafcfacbbccbfeebdeebee fccccdcfdbbeba ff ba f f fcbbebdccfcee eaf b cebabe a dbbebeeafababccefad cbbce feafcfacbbdfeecfcfeae bace cfadd cd fc afd bc bcadeaddfeecaf c ffebaa c bde cdecfa caeabeecdb  c  ffa faad afdcbe ffd dfcb ef e fdcbfaefec ccabbacac fecef dffd d afbdafebe bad bacbd daaefbe deabf  dcefbbcdfdbaf efdbabddffefabbcdb bbfffdbce e ee dcaaecfff faceaca  b bdb  aafaedccfbecbfd bedfaecdfffbfdc eceeebeccd  bf b bbfeb edaf fb

----
bbbbb cccc dcdd eeeeeff  ab bb bc d  eeeeeeffffffffff aa bbbbbbbbbb cccccccd d ee e aaaaaacbbcaccdddddceceeeffee a bbcccccccccd eeeeeeef ffaaaaaaa bbbbb ccccadddddddeeee ffffff aaaaa bbbbb a cdddcdeedeeeffffffa aaaacbbb  dddddd e eecbbbbbbbbbbcddddd  e fffaeaaaaac bdddddddeeedffff  a bbbbbccccc dddddd eeee f aaaaaabbbbbbb cccccc ddddeeee f a   ccccccccdddddeeeee   aabbbb ccccc dddde eee f aaabbbbb ccd  eeee fffffff aaa bbb  ccccccccddd e e  aaa bbbbbbbb cccc dd eeeeeefffffffaaaaaaa bbb cc c ddde ef ffff aa bbbbbbbb cd  deeefdf aaaaaa bb  b dddddee e fb bbbbbbbbbbba cccccddddd ee ffffffaf afbbbbb  ccccddd  eeef aaaaa bbbbbbbb cdd  eeeee  aaaaaabbb  ccccc  e eeeeffffff aaaaaba d ddd ee effeaaaaaaac cccccdddcd eeedeffff aaabab bdd  eeeeeea f bbbbbb  c   eee fe a bbbbb b  ddddddeeeeed afa b cccccccc ddddd ee fdf aa bbbbbbbbc cccedddddfeee feaaaaaaa bb ccccc  dd eeee fffaffff aaaaa cc ddcfdd e  a bab d dddddeee fffffffffffffffff aaa bbbbb dcccddc  eee ff aaaaa bbbbbbbbcccccccd e  effff

In [14]:
# GRU with Minimal Gated Units with Residual (adding h one step back) - seems to converge a bit faster
import numpy as np
import os.path

# Seed random
np.random.seed(0)

# Read data and setup maps for integer encoding and decoding.
data = open('input.txt', 'r').read()
chars = sorted(list(set(data))) # Sort makes model predictable (if seeded).
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # dict from enumerate: char and index
ix_to_char = { i:ch for i,ch in enumerate(chars) } # dict from enumerate: index and char

# Activation functions
# NOTE: Derivatives are calculated using outcomes of their primitives (which are already calculated during forward prop).
def sigmoid(input, deriv=False):
    if deriv:
        return input*(1-input) # sig' = sig(1-sig) - only if claculated in forward prop already!
    else:
        return 1 / (1 + np.exp(-input))

def tanh(input, deriv=False):
    if deriv:
        return 1 - input ** 2 # tanh'=1-tanh**2
    else:
        return np.tanh(input)

# Derivative is directly calculated in backprop (in combination with cross-entropy loss function).
def softmax(input):
    # Subtraction of max value improves numerical stability.
    e_input = np.exp(input - np.max(input))
    return e_input / e_input.sum()

# Hyper parameters
N, h_size, o_size = vocab_size, vocab_size, vocab_size # Hidden size is set to vocab_size, assuming that level of abstractness is approximately proportional to vocab_size (but can be set to any other value).
seq_length = 5 # Longer sequence lengths allow for lengthier latent dependencies to be trained.
learning_rate = 1e-1

# Model parameter initialization
#if os.path.isfile("gru_weights.data"):
    #initfile = open("gru_weights.data", "rb")
    #init_data = np.load(initfile)
    #Wy = init_data['Wy']
    #Wh = init_data['Wh']
    #Wr = init_data['Wr']
    #Wz = init_data['Wz']
    #Uh = init_data['Uh']
    #Ur = init_data['Ur']
    #Uz = init_data['Uz']
    #by = init_data['by']
    #bh = init_data['bh']
    #br = init_data['br']
    #bz = init_data['bz']
    #initfile.close()
#else:
Wz = np.random.rand(h_size, N) * 0.1 - 0.05
Uz = np.random.rand(h_size, h_size) * 0.1 - 0.05
bz = np.zeros((h_size, 1))

Wr = np.random.rand(h_size, N) * 0.1 - 0.05
Ur = np.random.rand(h_size, h_size) * 0.1 - 0.05
br = np.zeros((h_size, 1))

Wh = np.random.rand(h_size, N) * 0.1 - 0.05
Uh = np.random.rand(h_size, h_size) * 0.1 - 0.05
bh = np.zeros((h_size, 1))

Wy = np.random.rand(o_size, h_size) * 0.1 - 0.05
by = np.zeros((o_size, 1))

def lossFun(inputs, targets, hprev):
    # Initialize variables
    x, z, r, h_hat, h, y, p = {}, {}, {}, {}, {-1: hprev}, {}, {} # Dictionaries contain variables for each timestep.
    sequence_loss = 0

    # Forward prop
    for t in range(len(inputs)):
        # Set up one-hot encoded input
        x[t] = np.zeros((vocab_size, 1)) # for each time step t a one-hot over vocabulary size - init with all 0
        x[t][inputs[t]] = 1 # set t'th input to one (current word in one-hot)
        
        # Calculate update and reset gates
        if t>1:
            z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + 0.5*np.dot(Uz, h[t-2]) + bz) # update gate
            r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + 0.5*np.dot(Ur, h[t-2]) + br) # reset gate
        else:
            z[t] = sigmoid(np.dot(Wz, x[t]) + np.dot(Uz, h[t-1]) + bz) # update gate
            r[t] = sigmoid(np.dot(Wr, x[t]) + np.dot(Ur, h[t-1]) + br) # reset gate
        
        # Calculate hidden units
        if t>1:
            h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + 0.5*np.dot(Uh, np.multiply(r[t], h[t-2])) + bh)
            h[t] = np.multiply(z[t], h[t-1]) + 0.5*np.multiply(z[t], h[t-2]) + np.multiply((1 - z[t]), h_hat[t]) # sometimes denoted s[t]
        else: 
            h_hat[t] = tanh(np.dot(Wh, x[t]) + np.dot(Uh, np.multiply(r[t], h[t-1])) + bh)
            h[t] = np.multiply(z[t], h[t-1]) + np.multiply((1 - z[t]), h_hat[t]) # sometimes denoted s[t]
        
        # Regular output unit
        if t>1:
            y[t] = np.dot(Wy, h[t]) + 0.5*np.dot(Wy, h[t-1]) + by
        else:
            y[t] = np.dot(Wy, h[t]) + by
        
        # Probability distribution
        p[t] = softmax(y[t])
        
        # Cross-entropy loss
        # targets[t]'s entry of p[t]: since the output is 0 or 1 only one entry contributes (and yj*log(pj) becomes just log(pj))
        #loss = -np.sum(np.log(p[t][targets[t]])) # dict p: or time t (=one-hot position), whats the corresponsing target value?
        loss = -np.log(p[t][targets[t]])
        sequence_loss += loss

    # Parameter gradient initialization
    dWy, dWh, dWr, dWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
    dUh, dUr, dUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
    dby, dbh, dbr, dbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
    dhnext = np.zeros_like(h[0])
    
    # Backward prop
    for t in reversed(range(len(inputs))): # index t counting down
        # ∂loss/∂y
        dy = np.copy(p[t]) # copy output
        dy[targets[t]] -= 1 # the current target (truth) is 1 for the current t (an 0 for all other t's)
        
        # ∂loss/∂Wy and ∂loss/∂by
        if t>1:
            dWy += np.dot(dy, h[t].T) + 0.5*np.dot(dy, h[t-1].T)
        else:
            dWy += np.dot(dy, h[t].T) # weight updates: Wy -> Wy - etha * d loss / dWy -> dWy += etha * d loss / dWy
        dby += dy # weight for bias is just 1
        
        # Intermediary derivatives
        dh = np.dot(Wy.T, dy) + dhnext
        dh_hat = np.multiply(dh, (1 - z[t]))
        dh_hat_l = dh_hat * tanh(h_hat[t], deriv=True)
        
        # ∂loss/∂Wh, ∂loss/∂Uh and ∂loss/∂bh
        dWh += np.dot(dh_hat_l, x[t].T)
        if t>1:
            dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T) + 0.5*np.dot(dh_hat_l, np.multiply(r[t], h[t-2]).T)
        else:
            dUh += np.dot(dh_hat_l, np.multiply(r[t], h[t-1]).T)
        dbh += dh_hat_l # weight for bias is just 1
        
        # Intermediary derivatives
        drhp = np.dot(Uh.T, dh_hat_l)
        if t>1:
            dr = np.multiply(drhp, h[t-1]) + 0.5*np.multiply(drhp, h[t-2])
        else:
            dr = np.multiply(drhp, h[t-1]) 
        dr_l = dr * sigmoid(r[t], deriv=True) 
        
        # ∂loss/∂Wr, ∂loss/∂Ur and ∂loss/∂br
        dWr += np.dot(dr_l, x[t].T)
        if t>1:
            dUr += np.dot(dr_l, h[t-1].T) + 0.5*np.dot(dr_l, h[t-2].T) 
        else:
            dUr += np.dot(dr_l, h[t-1].T) 
        dbr += dr_l
        
        # Intermediary derivatives
        if t>1:
            dz = np.multiply(dh, h[t-1] - h_hat[t]) + 0.5*np.multiply(dh, h[t-2] - h_hat[t])
        else:
            dz = np.multiply(dh, h[t-1] - h_hat[t]) 
        dz_l = dz * sigmoid(z[t], deriv=True) 
        
        # ∂loss/∂Wz, ∂loss/∂Uz and ∂loss/∂bz
        dWz += np.dot(dz_l, x[t].T)
        if t>1:
            dUz += np.dot(dz_l, h[t-1].T) + 0.5*np.dot(dz_l, h[t-2].T)
        else:
            dUz += np.dot(dz_l, h[t-1].T) 
        dbz += dz_l 
        
        # All influences of previous layer to loss
        dh_fz_inner = np.dot(Uz.T, dz_l) 
        dh_fz = np.multiply(dh, z[t]) 
        dh_fhh = np.multiply(drhp, r[t]) 
        dh_fr = np.dot(Ur.T, dr_l) 
        
        # ∂loss/∂h𝑡₋₁
        dhnext = dh_fz_inner + dh_fz + dh_fhh + dh_fr

    return sequence_loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, h[len(inputs) - 1]

def sample(h, seed_ix, n):
    # Initialize first word of sample ('seed') as one-hot encoded vector.
    x = np.zeros((vocab_size, 1)) # np.zeros(shape) -> (vocab_size, 1) = (rows, cols)
    x[seed_ix] = 1
    ixes = [seed_ix]
    
    for t in range(n):
        # Calculate update and reset gates
        z = sigmoid(np.dot(Wz, x) + np.dot(Uz, h) + bz)
        r = sigmoid(np.dot(Wr, x) + np.dot(Ur, h) + br)
        
        # Calculate hidden units
        h_hat = tanh(np.dot(Wh, x) + np.dot(Uh, np.multiply(r, h)) + bh)
        h = np.multiply(z, h) + np.multiply((1 - z), h_hat)
        
        # Regular output unit
        y = np.dot(Wy, h) + by
        
        # Probability distribution
        p = softmax(y)

        # Choose next char according to the distribution
        ix = np.random.choice(range(vocab_size), p=p.ravel()) # ravel returns flattened array, P are the probabilities for choice
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

# Initialize sampling parameters and memory gradients (for adagrad)
n, p = 0, 0
mdWy, mdWh, mdWr, mdWz = np.zeros_like(Wy), np.zeros_like(Wh), np.zeros_like(Wr), np.zeros_like(Wz)
mdUh, mdUr, mdUz = np.zeros_like(Uh), np.zeros_like(Ur), np.zeros_like(Uz)
mdby, mdbh, mdbr, mdbz = np.zeros_like(by), np.zeros_like(bh), np.zeros_like(br), np.zeros_like(bz)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

print_interval = 100
n_runs = 1000

while True:
    # Reset memory if appropriate
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h_size, 1))
        p = 0 # current position
    
    # Get input and target sequence - each an index list of characters
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] # current position to sequence length
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] # current position+1 to sequence length+1 (next state)

    # Occasionally sample from model and print result
    if n % print_interval == 0:
        sample_ix = sample(hprev, inputs[0], 1000)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n%s\n----' % (txt, ))

    # Get gradients for current model based on input and target sequences
    loss, dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Occasionally print loss information
    if n % print_interval == 0:
        print('iter %d, loss: %f, smooth loss: %f' % (n, loss, smooth_loss))

    # Update model with adagrad (stochastic) gradient descent
    for param, dparam, mem in zip([Wy,  Wh,  Wr,  Wz,  Uh,  Ur,  Uz,  by,  bh,  br,  bz], # zip combines iterables
                                  [dWy, dWh, dWr, dWz, dUh, dUr, dUz, dby, dbh, dbr, dbz],
                                  [mdWy,mdWh,mdWr,mdWz,mdUh,mdUr,mdUz,mdby,mdbh,mdbr,mdbz]):
        np.clip(dparam, -5, 5, out=dparam) # limit values of array - here the changes (dparam has the dWy,...)
        mem += dparam * dparam # store the squared change (gradient)
        # now actually change the model parameters
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # Small added term for numerical stability

    # Prepare for next iteration
    p += seq_length
    n += 1
    
    # check exit
    if n>n_runs:
        break
        
# save weights
#outfile = open("gru_weights.data", "wb")
#outdata = dict()
#outdata['Wy'] = Wy
#outdata['Wh'] = Wh
#outdata['Wr'] = Wr
#outdata['Wz'] = Wz
#outdata['Uh'] = Uh
#outdata['Ur'] = Ur
#outdata['Uz'] = Uz
#outdata['by'] = by
#outdata['bh'] = bh
#outdata['br'] = br
#outdata['bz'] = bz
#np.savez(outfile, **outdata)
#outfile.close()

data has 576 characters, 7 unique.
----
a e cbdfd cccda bfafccfcebfddbedaaefcdfcfdefeadb bae fbb abf ffcbad  c aaa   dffbadcf ebadadcbcbdbbef abefffbfa fad efabbeaacbeacfdffedffc bab dbffcadfbcacbacbbcebff ac bedddabdfca d bcf b bdaa dddf dddf decbe ccd abdfd ffcfcfeff abfbafcfadc da   fefdcbfbcbfabd dcaacbcbf  bddbfbedeffedba accef  deeffbbaddf cdaaf c cfbf abbdbddbcd edec bbb e dddf f cbcba ceba  a a  eacecaaccbeef fbfdfebcbbacdac a f ad fefcbcfcafb abfccbfffbfedaaafbccee bee ab b cad ffbaac dbbedfbfebdbf aabecc b afafead edecf beba ecfebbcfe dfdac bffabdfaafcfacbbccbfeebdeebee fccccdcfdbbeba ff ba f f fcbbebdccfcee eaf b cebabe a dbbebeeafababccefad cbbce feafcfacbbdfeecfcfeae bace cfadd cd fc afd bc bcadeaddfeecaf c ffebaa c bde cdecfa caeabeecdb  c  ffa faad afdcbe ffd dfcb ef e fdcbfaefec ccabbacac fecef dffd d afbdafebe bad bacbd daaefbe deabf  dcefbbcdfdbaf efdbabddffefabbcdb bbfffdbce e ee dcaaecfff faceaca  b bdb  aafaedccfbecbfd bedfaecdfffbfdc eceeebeccd  bf b bbfeb edaf fb

----
bbcccccbb bb ccbbccccccb cd df bc caaaacbbbbabbbccccc dd effef bbbbbbbbbbabb cbdc dbeeff afaaba bbbccdcd efeff aaa bbdedded fff fcfe bbc ccbbbbbbb cccccccbdd eeeeeaaa bbbbbbbbb ccccc cbdb aa abbbbbbb ccccccccccbccccdddd ddd fff b bbbbbbcccccccdfdddd cf fff baaaa cbbcccdcdddd cffea ababbbbbddd d febabb bbbb cad dcfbeab bbb bcccccccdddeeee faa a cbbbbbbbccbddeeeee aaaaccccbcccccbb ccdbeec fabbbcccccbccd cfd caacbbbbc cdd ddc aaabbbbbbbbb cbb ccccbd deceff baac ccacccccccccccccccccccccccc dd dadbab ccbddee df ccbbbbbbbce bbdc fcd eaaaaa bb bbbbbbccccbccccbcd dde ebcbbabbbbbbcccc ccbddeefe f afbbbbb bbbb cdc cceff aaaaa bbbbbcccbdfe aaabab bbcccccdcc befeff af bbbbbbf cc cccccdb f cfaabbaaba cccccccdcebeeeeaa babbbbbbbbc cccd dcaf aaaaaba bbcbbbb ccb c  adef fa aaaa bb ba ccccccdccccc cdd e fffff aaabbbbb bb ccc ccbe eefffff aaafa bbbdccccccccccccc ccccccccb ccccccc cdc eeff abbbb cb ccbf cbba cbcbb ebb cdddd cfefeaa bbbbbbccccccccbbbcde ffdaff aaaaa bbbcccccccccddedef faaaaa bb bcccc